# Kubernetes & Tensorflow Serving for Image Classification

- Two tier architecture
- TF-Serving - GPU intensive, C++
- Gateway
  - process user input from website, input to TF-Serving
  - process gRPC from TF Serving and output predictions in a valid format for the website

In [4]:
# !wget https://github.com/DataTalksClub/machine-learning-zoomcamp/releases/download/chapter7-model/xception_v4_large_08_0.894.h5 -O clothing-model.h5

In [7]:
# Convert the h5 to the format "saved_model"
import tensorflow as tf
from tensorflow import keras
model = keras.models.load_model("clothing-model-v4.h5")

In [8]:
# Model and folder in shihc the model will be saved
tf.saved_model.save(model, "clothing-model")

INFO:tensorflow:Assets written to: clothing-model/assets


INFO:tensorflow:Assets written to: clothing-model/assets


### Check the saved_model signature definition
- `saved_model_cli show --dir clothing-model --all`

```bash
signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_8'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 299, 299, 3)
        name: serving_default_input_8:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_7'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
```
- inputs.shape
  - `-1` - can take as arbitrarily many images
  - `299, 299, 3` - shape of a single image
- outputs.shape
  - `-1` - can be many outputs
  - `10` - single output shape
- summary of model description
  - `serving_default`
  - `input_8` - input
  - `dense_7` - output

### Can run TF-Serving locally now that these values are known
- Inject local folder with the model (`clothing-model` folder), into the Docker container where TF-Serving is running, using mounting Docker volumes (-v)
- Essentially maps folder from host machine to the container, similar syntax as with ports - `<host>:<container>`
- Need to add env variable with `MODEL_NAME` (should be same as the folder name in container to which the local folder is moounted) 
-  
```bash
docker run -it --rm \
 -p 8500:8500 \
 -v "./clothing-model:/models/clothing-model/1" \
 -e MODEL_NAME="clothing-model" \
 tensorflow/serving:2.7.0
```
- Testing in Notebook `tf_serving_connect`

## 10.3 Creating a pre-processing service _aka_ Gateway
- Converting the notebook to a Python script
- Wrapping the script into a Flask app
- Getting rid of the Tensorflow dep with `tensorflow-protobuf` - see file `proto.py`, as we're only using one function and the lib is too huge